# 初期設定

In [30]:
import pandas as pd
import numpy as np
import gensim
from scipy.spatial import distance
from gensim.models import word2vec 

import logging

import sys
import MeCab

In [31]:
#snssns..set_styleset_sty ('whitegrid')
%matplotlib inline

pd.set_option("display.max_columns", 200) # 表示カラムの最大値
pd.set_option('display.max_rows', 200) # 表示行数の最大値
pd.set_option('display.max_colwidth', 30)
pd.set_option("display.float_format", '{:.3f}'.format) # 小数点以下の有効数字

# jupyterの幅を自動調整
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## モデル読み込み

In [49]:
from gensim.models.word2vec import Word2Vec

model_path = '../../02.model/word2vec.gensim.model'
model = Word2Vec.load(model_path)

# LexRank

参照：https://qiita.com/takumi_TKHS/items/4a56ac151c60da8bde4b

In [39]:
text = """米カリフォルニア州クパチーノの本社で開いた発表会で最大の目玉となったのは、定額制の動画配信サービス「アップルTV＋（プラス）」だ。
アップルも資金を出して著名な映画監督や俳優らと連携し、他の動画配信サービスでは見られない独自の映画や番組、ドキュメンタリーを制作する。会場にはスティーブン・スピルバーグ監督らも駆けつけ、制作への意欲を語った。
「アップルTV＋」はiPhoneなど同社製品の利用者を主なターゲットに100を超す国や地域で提供する。
ティム・クック最高経営責任者（CEO）は「創造性豊かな人々を結集し、これまでにない新しいサービスを作り上げる」と語った。
料金など詳細はサービス開始直前に発表するとの説明にとどめた。
アップルは動画配信分野では後発だ。専業の米ネットフリックスが先行し、27言語に対応する配信体制に強みがある。契約数は世界で1億4000万件にのぼる。
既存のコンテンツ企業も加わって競争が激しくなる見込みで、米ウォルト・ディズニーが19年後半にスタートさせる。アップルは世界で14億台を超える製品が強みとなる。
同社は定額制のニュース配信サービス「アップルニュース＋」を同日から米国で始めた。
専用のアプリケーションをダウンロードし、月9.99ドル（約1100円）を払えば経済紙「ウォール・ストリート・ジャーナル」や経済誌「フォーチュン」など300を超える新聞・雑誌が提供する記事を好きなだけ読める。
有力誌「タイム」や「ニューヨーカー」、ファッション誌「ヴォーグ」、科学誌「ナショナルジオグラフィック」などが参加する。
各コンテンツをそれぞれ個別に購入した場合は年8000ドルに相当するという。
当初は米国とカナダでサービスを始め、オーストラリアや欧州に段階的にサービス地域を拡大する。
アップルは同日、100を超える新作を定額で遊べるゲーム配信サービス「アップルアーケード」を19年秋に始めることも明らかにした。
iPhone上の決済サービスの利便性を高めるため、米金融大手ゴールドマン・サックスなどと組んで独自のクレジットカード「アップルカード」を19年夏から米国で発行する計画も表明した。"""

# 1行1文となっているため、改行コードで分離
sentences = [t for t in text.split('\n')]
for i in range(2):
    print(sentences[i])

米カリフォルニア州クパチーノの本社で開いた発表会で最大の目玉となったのは、定額制の動画配信サービス「アップルTV＋（プラス）」だ。
アップルも資金を出して著名な映画監督や俳優らと連携し、他の動画配信サービスでは見られない独自の映画や番組、ドキュメンタリーを制作する。会場にはスティーブン・スピルバーグ監督らも駆けつけ、制作への意欲を語った。


In [72]:
# テキストのベクトルを計算
def compute_word2vec(text):
    mt = MeCab.Tagger('')
    mt.parse('')
    sum_vec = np.zeros(model.vector_size)
    word_count = 0
    node = mt.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        # 名詞、動詞、形容詞に限定
        if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞':
            sum_vec += model.wv[node.surface]
            word_count += 1
        node = node.next

    return sum_vec / word_count

In [88]:
compute_word2vec('text')

KeyError: "word 'ウォール・ストリート・ジャーナル' not in vocabulary"

In [90]:
mt = MeCab.Tagger('')
mt.parse('')
sum_vec = np.zeros(model.vector_size)
word_count = 0
node = mt.parseToNode(text)

while node:
    fields = node.feature.split(",")
    # 名詞、動詞、形容詞に限定
    if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞':
        print(1)
        
#         sum_vec += model.wv[node.surface]
#         word_count += 1
#     node = node.next

# return sum_vec / word_count

KeyboardInterrupt: 

In [73]:
def compute_cosine(v1, v2):
    return 1 - distance.cosine(v1, v2)

In [74]:
def PowerMethod(CosineMatrix, N, err_tol):
    p_old = np.array([1.0/N]*N)
    err = 1

    while err > err_tol:
        err = 1
        p = np.dot(CosineMatrix.T, p_old)
        err = np.linalg.norm(p - p_old)
        p_old = p

    return p

In [75]:
N = 100
np.zeros(N)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [76]:
def lexrank(sentences, N, threshold, vectorizer):
    CosineMatrix = np.zeros([N, N])
    degree = np.zeros(N)
    L = np.zeros(N)

    vector = compute_word2vec(sentences)

    # Computing Adjacency Matrix                                                                                                                                         
    for i in range(N):
        for j in range(N):
            CosineMatrix[i,j] = compute_cosine(vector[i], vector[j])
            if CosineMatrix[i,j] > threshold:
                CosineMatrix[i,j] = 1
                degree[i] += 1
            else:
                CosineMatrix[i,j] = 0

    # Computing LexRank Score                                                                                                                                            
    for i in range(N):
        for j in range(N):
            CosineMatrix[i,j] = CosineMatrix[i,j] / degree[i]

    L = PowerMethod(CosineMatrix, N, err_tol=10e-6)

    return L

In [77]:
lexrank(text,len(text),0.1,'word2vec')

KeyError: "word 'ウォール・ストリート・ジャーナル' not in vocabulary"